In [0]:
file_path = "dbfs:/FileStore/shared_uploads/ali.m@campus.technion.ac.il/summaries_train.csv" 


train = spark.read.format("csv").option("header", "true") \
    .option("delimiter", ",") \
    .option("multiline", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .option("encoding", "UTF-8")\
    .load(file_path)



In [0]:
train_sample = train.sample(False, 0.04, seed=85)

train_sample.count()

Out[2]: 926

In [0]:

!pip install torch transformers


  Using cached torch-2.2.2-cp39-cp39-manylinux1_x86_64.whl (755.5 MB)
  Using cached transformers-4.39.3-py3-none-any.whl (8.8 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached triton-2.2.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64

In [0]:
import pandas as pd
from transformers import GPT2Tokenizer
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# df = pd.read_parquet('/path_to_local_storage/job_posts_prepared.parquet')
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')

# Add a padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize data
inputs = tokenizer(train_sample.rdd.map(lambda x: x[0]).collect(), 
                   max_length=512, truncation=True, padding="max_length", return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

In [0]:
from torch.utils.data import Dataset, DataLoader

class JobPostDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, idx):
        return {key: tensor[idx] for key, tensor in self.encodings.items()}

dataset = JobPostDataset(inputs)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('distilgpt2')
model = model.to(device)  # Move model to GPU






model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

for epoch in range(3):  # number of epochs
    model.train()
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['input_ids'].to(device)  # GPT uses the same input as labels for language modeling
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch}, Loss: {loss.item()}")



Epoch: 0, Loss: 3.9932637214660645
Epoch: 0, Loss: 4.802980422973633
Epoch: 0, Loss: 4.686226844787598
Epoch: 0, Loss: 4.383626937866211
Epoch: 0, Loss: 4.175486087799072
Epoch: 0, Loss: 3.818389892578125
Epoch: 0, Loss: 4.2484049797058105
Epoch: 0, Loss: 4.03070592880249
Epoch: 0, Loss: 4.260434627532959
Epoch: 0, Loss: 3.865204334259033
Epoch: 0, Loss: 3.6774380207061768
Epoch: 0, Loss: 4.131802558898926
Epoch: 0, Loss: 4.09811544418335
Epoch: 0, Loss: 4.009454250335693
Epoch: 0, Loss: 4.157166004180908
Epoch: 0, Loss: 4.3007307052612305
Epoch: 0, Loss: 4.0141754150390625
Epoch: 0, Loss: 3.9246556758880615
Epoch: 0, Loss: 3.981076240539551
Epoch: 0, Loss: 3.9608993530273438
Epoch: 0, Loss: 4.152264595031738
Epoch: 0, Loss: 3.9835588932037354
Epoch: 0, Loss: 3.6435391902923584
Epoch: 0, Loss: 3.7954697608947754
Epoch: 0, Loss: 3.9234023094177246
Epoch: 0, Loss: 3.692347288131714
Epoch: 0, Loss: 3.782850980758667
Epoch: 0, Loss: 3.9707233905792236
Epoch: 0, Loss: 4.017848491668701
Epoc

In [0]:
model.save_pretrained('dbfs:/FileStore/shared_uploads/ali.m@campus.technion.ac.il/jobs_gpt')

In [0]:
# max_length: The maximum length of the sequence to be generated.
# num_return_sequences: Number of sequences to return.
# no_repeat_ngram_size: Ensures no n-gram appears twice.
# repetition_penalty: The penalty for repeating the same line. Higher is more strict.
# top_p: Controls diversity via nucleus sampling: top p tokens are considered.
# temperature: The higher the temperature, the more random the output. Closer to zero makes it more deterministic.
# do_sample: Whether or not to use sampling; use True for diverse generation.
# top_k: The number of highest probability vocabulary tokens to keep for top-k-filtering.
# early_stopping: Whether to stop the beam search when at least num_beams sentences are finished per batch.

def generate_text(prompt, max_length=100):
    # Encode the input prompt to get the token ids
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_ids = input_ids.to(device)

    # Generate text using the model
    outputs = model.generate(
        input_ids, 
        max_length=max_length, 
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        repetition_penalty=2.5,
        top_p=0.92,
        temperature=0.85,
        do_sample=True,
        top_k=125,
        early_stopping=True
    )

    # Decode and print the output text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_text)

# Example prompt
prompt = "Technical requirements for the software developer position include"
generate_text(prompt, max_length=512)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-0b757462-1e89-43b8-8a45-bb1136bde608/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Technical requirements for the software developer position include but are not limited to:
We support a critical mission that is changing massively parallelism. We create an automated and lightning-fast internet, which provides hardware and networking infrastructure in all of our major business functions such as TCP/UDP routers or routers; we require users, engineers know ho

In [0]:
prompt = "write a job posting for sales representative"
generate_text(prompt, max_length=512)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-0b757462-1e89-43b8-8a45-bb1136bde608/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
write a job posting for sales representative in the market
of DEVA (Excel: Retail, General & Military) and International?
If you’re looking to make an immediate impact on our customers' lives while using your strong analytical skills as well taking pride of what matters most - if not exceeding their expectations. If that sounds like you, we want YOU TO BRING out with us! We 